# Trablho de Conclusão de Curso - UNIVESP - 1S/2025- Notebook1

# Código: tcc530-sala-002grupo-002

## 1.0 Packges

### Packges Import

In [1]:
# Packages

# Bibliotecas Gerais
import numpy                   as np
import pandas                  as pd
import matplotlib.pyplot       as plt
import seaborn                 as sns
import sys
import time
from scipy                     import stats

#Bibliotecas de Regressão
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing   import StandardScaler
from sklearn.svm             import SVR
from sklearn.tree            import DecisionTreeRegressor
from sklearn.linear_model    import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, ElasticNetCV, HuberRegressor, LassoLars, BayesianRidge
from sklearn.linear_model    import SGDRegressor
from sklearn.ensemble        import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor

# Bibliotecas de Clusterização
import scipy.cluster.hierarchy as shc
from sklearn.cluster           import KMeans
from sklearn.metrics           import silhouette_score, davies_bouldin_score,v_measure_score
from sklearn.mixture           import GaussianMixture
from sklearn.neural_network    import MLPRegressor
from sklearn.metrics           import r2_score, mean_absolute_error,mean_squared_error
from sklearn.model_selection   import GridSearchCV
from sklearn.model_selection   import RandomizedSearchCV
from sklearn.neighbors         import NearestNeighbors
from sklearn.cluster           import DBSCAN
from sklearn.cluster           import AgglomerativeClustering

# Bibliotecas Classificação
from sklearn                       import metrics
from sklearn.preprocessing         import MinMaxScaler
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.linear_model          import LogisticRegression
from sklearn.tree                  import DecisionTreeClassifier
from sklearn.ensemble              import RandomForestClassifier
from sklearn.naive_bayes           import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB
from sklearn.svm                   import SVC
from sklearn.neural_network        import MLPClassifier
from sklearn.metrics               import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics               import classification_report
from sklearn.metrics               import ConfusionMatrixDisplay
from sklearn.metrics               import RocCurveDisplay

# Bibliotecas de Recomendação
from sklearn.metrics.pairwise      import cosine_similarity

# Redução de Dimensionalidade
from sklearn.decomposition         import PCA
from sklearn.decomposition         import TruncatedSVD

### Packges Version Control

In [2]:
pip freeze --local > ../requeriments.txt

Note: you may need to restart the kernel to use updated packages.


## 2.0 Data Read

In [3]:
link   = '../dados/dados_filtrados_2014.csv'
df2014 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2015.csv'
df2015 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2016.csv'
df2016 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2017.csv'
df2017 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2018.csv'
df2018 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2019.csv'
df2019 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2020.csv'
df2020 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2021.csv'
df2021 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2022.csv'
df2022 = pd.read_csv(link,sep=",",engine='python')

link   = '../dados/dados_filtrados_2023.csv'
df2023 = pd.read_csv(link,sep=",",engine='python')

## 3.0 Data Join

In [4]:
df2021 = df2021.rename(columns={"ANO_NASC": "DT_NASC"})
df2022 = df2022.rename(columns={"ANO_NASC": "DT_NASC"})
df2023 = df2023.rename(columns={"ANO_NASC": "DT_NASC"})

In [5]:
df_full = pd.concat([df2014, df2015,df2016,df2017,df2018,df2019,df2020,df2021,df2022,df2023], axis=0, ignore_index=True)

In [6]:
def verificacao_nulos(df_true,target_clean,col_ok):

  df = df_true.copy()

  target_clean_por = 100*target_clean

  names_df  = df.columns
  nnames_df = len(names_df)
  porcent_df_null = df.isnull().sum()/df.shape[0]

  for i in range(0,nnames_df):
    
      if(porcent_df_null[i]>target_clean):

          print('A feature %s possui mais do que %.2f porcento de dados nulos'%(names_df[i],target_clean_por))

      else:

          col_ok.append(names_df[i])
          

In [7]:
target_clean = 0.5
col_ok       = []
verificacao_nulos(df_full,target_clean,col_ok)

A feature ID_OCUPA_N possui mais do que 50.00 porcento de dados nulos
A feature DT_CHIK_S1 possui mais do que 50.00 porcento de dados nulos
A feature DT_CHIK_S2 possui mais do que 50.00 porcento de dados nulos
A feature DT_PRNT possui mais do que 50.00 porcento de dados nulos
A feature RES_CHIKS1 possui mais do que 50.00 porcento de dados nulos
A feature RES_CHIKS2 possui mais do que 50.00 porcento de dados nulos
A feature RESUL_PRNT possui mais do que 50.00 porcento de dados nulos
A feature DT_SORO possui mais do que 50.00 porcento de dados nulos
A feature DT_NS1 possui mais do que 50.00 porcento de dados nulos
A feature DT_VIRAL possui mais do que 50.00 porcento de dados nulos
A feature DT_PCR possui mais do que 50.00 porcento de dados nulos
A feature SOROTIPO possui mais do que 50.00 porcento de dados nulos
A feature DT_INTERNA possui mais do que 50.00 porcento de dados nulos
A feature UF possui mais do que 50.00 porcento de dados nulos
A feature MUNICIPIO possui mais do que 50.00 p

/tmp/ipykernel_88554/923589582.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(porcent_df_null[i]>target_clean):


In [8]:
df_clean = df_full[col_ok].dropna(axis=0).reset_index(drop=True)
df_clean.head(5)

,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,RESUL_PCR_,HISTOPA_N,IMUNOH_N,HOSPITALIZ,CLASSI_FIN,CRITERIO,EVOLUCAO,DT_ENCERRA,TP_SISTEMA,CS_FLXRET
0,2,A90,2017-01-24,201704.0,2017,35,354990,1351.0,3507157.0,2017-01-21,...,4.0,4.0,4.0,2.0,10.0,1.0,1.0,2017-01-24,2.0,1.0
1,2,A90,2017-07-12,201728.0,2017,35,355030,1331.0,2080818.0,2017-06-11,...,4.0,4.0,4.0,2.0,10.0,2.0,1.0,2017-09-11,2.0,1.0
2,2,A90,2017-01-29,201705.0,2017,35,352390,1353.0,2085437.0,2017-01-28,...,4.0,4.0,4.0,2.0,5.0,2.0,1.0,2017-04-10,2.0,1.0
3,2,A90,2017-03-27,201713.0,2017,35,351870,1349.0,6885284.0,2017-03-15,...,4.0,4.0,4.0,2.0,5.0,1.0,1.0,2017-05-22,2.0,1.0
4,2,A90,2017-01-23,201704.0,2017,35,350280,1336.0,2043890.0,2017-01-23,...,4.0,4.0,4.0,2.0,5.0,1.0,1.0,2017-01-31,2.0,1.0


In [9]:
reduct = 100*(1-(df_clean.shape[0]/df_full.shape[0]))
print('Full Size: %d'%df_full.shape[0])
print('Clean Size: %d'%df_clean.shape[0])
print('Reduction in Size: %f'%reduct)

Full Size: 4199371
Clean Size: 552333
Reduction in Size: 86.847245


In [10]:
df_clean.columns

Index(['TP_NOT', 'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT',
       'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI',
       'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
       'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
       'DT_INVEST', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO',
       'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA',
       'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO', 'DIABETES',
       'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT',
       'AUTO_IMUNE', 'RESUL_SORO', 'RESUL_NS1', 'RESUL_VI_N', 'RESUL_PCR_',
       'HISTOPA_N', 'IMUNOH_N', 'HOSPITALIZ', 'CLASSI_FIN', 'CRITERIO',
       'EVOLUCAO', 'DT_ENCERRA', 'TP_SISTEMA', 'CS_FLXRET'],
      dtype='object')

In [11]:
df_clean = df_clean.ffill()

/tmp/ipykernel_88554/814116313.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean = df_clean.ffill()


In [12]:
target_clean = 0.5
col_ok       = []
verificacao_nulos(df_clean,target_clean,col_ok)

/tmp/ipykernel_88554/923589582.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(porcent_df_null[i]>target_clean):


In [23]:
link   = '../dados/municipios_sp_edited.csv'
df_muni_sp = pd.read_csv(link,sep=",")
df_muni_sp = df_muni_sp.drop(['ID_ESTADO','ESTADO'],axis=1)
df_muni_sp.head()

,REGIAO,MUNICIPIO,ID_MUNICIPIO
0,Presidente Prudente,Adamantina,350010
1,São José do Rio Preto,Adolfo,350020
2,Campinas,Aguaí,350030
3,Campinas,Águas da Prata,350040
4,Campinas,Águas de Lindóia,350050


In [24]:
df_muni_sp = df_muni_sp.rename(columns={"ID_MUNICIPIO": "ID_MUNICIP"})

In [25]:
df_clean = df_clean.merge(df_muni_sp,on=['ID_MUNICIP'])
df_clean.head(5)

,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,CRITERIO,EVOLUCAO,DT_ENCERRA,TP_SISTEMA,CS_FLXRET,REGIAO_x,MUNICIPIO_x,DT_NASC_YEAR,REGIAO_y,MUNICIPIO_y
0,2,A90,2017-01-24,201704.0,2017,35,354990,1351.0,3507157.0,2017-01-21,...,1.0,1.0,2017-01-24,2.0,1.0,São José dos Campos,São José dos Campos,1970-01-01 00:00:00.000001974,São José dos Campos,São José dos Campos
1,2,A90,2017-07-12,201728.0,2017,35,355030,1331.0,2080818.0,2017-06-11,...,2.0,1.0,2017-09-11,2.0,1.0,São Paulo,São Paulo,1970-01-01 00:00:00.000002009,São Paulo,São Paulo
2,2,A90,2017-01-29,201705.0,2017,35,352390,1353.0,2085437.0,2017-01-28,...,2.0,1.0,2017-04-10,2.0,1.0,Sorocaba,Itu,1970-01-01 00:00:00.000002014,Sorocaba,Itu
3,2,A90,2017-03-27,201713.0,2017,35,351870,1349.0,6885284.0,2017-03-15,...,1.0,1.0,2017-05-22,2.0,1.0,São Paulo,Guarujá,1970-01-01 00:00:00.000001965,São Paulo,Guarujá
4,2,A90,2017-01-23,201704.0,2017,35,350280,1336.0,2043890.0,2017-01-23,...,1.0,1.0,2017-01-31,2.0,1.0,Araçatuba,Araçatuba,1970-01-01 00:00:00.000001975,Araçatuba,Araçatuba


In [16]:
df_clean.dtypes

TP_NOT          int64
ID_AGRAVO      object
DT_NOTIFIC     object
SEM_NOT       float64
NU_ANO          int64
SG_UF_NOT       int64
ID_MUNICIP      int64
ID_REGIONA    float64
ID_UNIDADE    float64
DT_SIN_PRI     object
SEM_PRI         int64
DT_NASC        object
NU_IDADE_N    float64
CS_SEXO        object
CS_GESTANT    float64
CS_RACA       float64
CS_ESCOL_N    float64
SG_UF         float64
ID_MN_RESI    float64
ID_RG_RESI    float64
ID_PAIS       float64
DT_INVEST      object
FEBRE         float64
MIALGIA       float64
CEFALEIA      float64
EXANTEMA      float64
VOMITO        float64
NAUSEA        float64
DOR_COSTAS    float64
CONJUNTVIT    float64
ARTRITE       float64
ARTRALGIA     float64
PETEQUIA_N    float64
LEUCOPENIA    float64
LACO          float64
DOR_RETRO     float64
DIABETES      float64
HEMATOLOG     float64
HEPATOPAT     float64
RENAL         float64
HIPERTENSA    float64
ACIDO_PEPT    float64
AUTO_IMUNE    float64
RESUL_SORO    float64
RESUL_NS1     float64
RESUL_VI_N

In [17]:
df_clean['DT_NASC_YEAR'] = pd.DatetimeIndex(df_clean['DT_NASC']).year
df_clean = df_clean.drop(['DT_NASC'],axis=1)

df_clean['DT_NOTIFIC']   = pd.to_datetime(df_clean['DT_NOTIFIC'])
df_clean['DT_SIN_PRI']   = pd.to_datetime(df_clean['DT_SIN_PRI'])
df_clean['DT_NASC_YEAR'] = pd.to_datetime(df_clean['DT_NASC_YEAR'])
df_clean['DT_INVEST']    = pd.to_datetime(df_clean['DT_INVEST'])
df_clean['DT_ENCERRA']   = pd.to_datetime(df_clean['DT_ENCERRA'])

df_clean['REGIAO']     = df_clean['REGIAO'].astype('category')
df_clean['MUNICIPIO']  = df_clean['MUNICIPIO'].astype('category')
df_clean['ID_AGRAVO']  = df_clean['ID_AGRAVO'].astype('category')
df_clean['ID_MUNICIP'] = df_clean['ID_MUNICIP'].astype('category')
df_clean['ID_REGIONA'] = df_clean['ID_REGIONA'].astype('category')
df_clean['ID_UNIDADE'] = df_clean['ID_UNIDADE'].astype('category')
df_clean['ID_MN_RESI'] = df_clean['ID_MN_RESI'].astype('category')
df_clean['ID_RG_RESI'] = df_clean['ID_RG_RESI'].astype('category')
df_clean['ID_PAIS']    = df_clean['ID_PAIS'].astype('category')
df_clean['CS_SEXO']    = df_clean['CS_SEXO'].astype('category')

df_clean['SEM_PRI']    = df_clean['SEM_PRI'].astype('float')
df_clean['ID_MN_RESI'] = df_clean['ID_MN_RESI'].astype('float')

In [18]:
df_clean.dtypes

TP_NOT                   int64
ID_AGRAVO             category
DT_NOTIFIC      datetime64[ns]
SEM_NOT                float64
NU_ANO                   int64
SG_UF_NOT                int64
ID_MUNICIP            category
ID_REGIONA            category
ID_UNIDADE            category
DT_SIN_PRI      datetime64[ns]
SEM_PRI                float64
NU_IDADE_N             float64
CS_SEXO               category
CS_GESTANT             float64
CS_RACA                float64
CS_ESCOL_N             float64
SG_UF                  float64
ID_MN_RESI             float64
ID_RG_RESI            category
ID_PAIS               category
DT_INVEST       datetime64[ns]
FEBRE                  float64
MIALGIA                float64
CEFALEIA               float64
EXANTEMA               float64
VOMITO                 float64
NAUSEA                 float64
DOR_COSTAS             float64
CONJUNTVIT             float64
ARTRITE                float64
ARTRALGIA              float64
PETEQUIA_N             float64
LEUCOPEN

In [19]:
muni_list = df_clean['MUNICIPIO'].unique().tolist()

In [20]:
regiao_list = df_clean['REGIAO'].unique().tolist()

In [21]:
df_clean.groupby(['REGIAO'])['RESUL_SORO'].sum()

/tmp/ipykernel_88554/2511805506.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_clean.groupby(['REGIAO'])['RESUL_SORO'].sum()


REGIAO
Araraquara               138123.0
Araçatuba                109625.0
Bauru                    205259.0
Campinas                 216132.0
Marília                   75595.0
Presidente Prudente      219681.0
Ribeirão Preto           187585.0
Sorocaba                  47290.0
São José do Rio Preto    236508.0
São José dos Campos       66583.0
São Paulo                138117.0
Name: RESUL_SORO, dtype: float64

In [22]:
df_clean.groupby(['MUNICIPIO'])['RESUL_SORO'].sum()

/tmp/ipykernel_88554/798060027.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_clean.groupby(['MUNICIPIO'])['RESUL_SORO'].sum()


MUNICIPIO
Adamantina            12008.0
Adolfo                  151.0
Aguaí                  3577.0
Agudos                 5968.0
Alambari                 41.0
                       ...   
Águas de São Pedro      117.0
Álvares Florence         75.0
Álvares Machado         989.0
Álvaro de Carvalho       39.0
Óleo                      4.0
Name: RESUL_SORO, Length: 563, dtype: float64

## 4.0 Data Transform

### 4.1 Save Clean Data Set

In [ ]:
link = '../dados/df_clean.csv'
df_clean.to_csv(link,sep=',',index=False)

## 5.0 Exploratory Data Analysis

In [ ]:
df_clean.describe()

In [ ]:
df_numeric = df_clean.select_dtypes(include='number')

In [ ]:
plt.figure(figsize=(20,12))
sns.heatmap(df_numeric.corr(), annot=True,cmap='BrBG')
plt.show()
plt.close()

In [ ]:
corr_matrix = df_numeric.corr()
corr_matrix

In [ ]:
colunas = df_numeric.select_dtypes(include='number').columns
ncolunas = len(colunas)

for i in range(0,ncolunas):
 plt.figure(figsize=(20,20))
 sns.displot(x=colunas[i],data=df_numeric)
 plt.show()
 plt.close()

## 6.0 Data Modeling

In [ ]:
# n_components         = 10
# pca                  = PCA(n_components=n_components)
# df_clean_reduzido   = pca.fit_transform(df_clean)
# df_clean_reduzido   = pd.DataFrame(df_clean_reduzido)

## 7.0 Data Preparing

## 8.0 ML Models

### 8.1 Traning

### 8.2 Validation

### 8.3 Measuring

### 8.4 Hyperparamerization

## 9.0 ML Models and Data Visualization